In [6]:
import os

data_dir = "hyperpartisan_data"

# Find all files in the data directory to see what was extracted
all_files = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        all_files.append(os.path.join(root, file))

print("Files in the data directory:")
for file in all_files:
    print(f"- {file}")

# Look specifically for XML files
xml_files = [f for f in all_files if f.endswith('.xml')]
print("\nXML files found:")
for xml_file in xml_files:
    print(f"- {xml_file}")

Files in the data directory:
- hyperpartisan_data/articles-training-byarticle-20181122.xml
- hyperpartisan_data/articles-training-byarticle.zip

XML files found:
- hyperpartisan_data/articles-training-byarticle-20181122.xml
